🧹 Preprocessing
Notebook ini digunakan untuk preprocessing data sebelum pemodelan.

Langkah yang disarankan:
- Load dataset
- Cek dan tangani missing value
- Encode variabel kategorikal
- Normalisasi atau standarisasi
